### **Alpaca API Python Wrapper**

In [3]:
import pandas as pd
import configparser
from datetime import datetime
import time
from alpaca.data import CryptoHistoricalDataClient, StockHistoricalDataClient
from alpaca.data import CryptoDataStream, StockDataStream
from alpaca.data.requests import StockLatestQuoteRequest, CryptoLatestQuoteRequest
from alpaca.data.requests import StockBarsRequest, CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame
from alpaca.trading.client import TradingClient
from alpaca.trading.client import TradingClient
from alpaca.trading.requests import GetAssetsRequest
from alpaca.trading.requests import MarketOrderRequest
from alpaca.trading.requests import LimitOrderRequest
from alpaca.trading.requests import StopOrderRequest
from alpaca.trading.requests import StopLimitOrderRequest
from alpaca.trading.requests import TrailingStopOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce
from alpaca.trading.enums import AssetClass
from alpaca.trading.enums import AssetStatus
from alpaca.trading.enums import AssetExchange
from typing import Any
#from tradeAlpaca import tradeAlpaca

In [4]:
class tradeAlpaca(object):
    '''python wrapper class for the Alpaca trading API via Alpaca-py'''

    def __init__(self, keys_file):
        '''
        Parameters
        ----------
        keys_file: configuration .cfg file path
        with the following text contained within
        that file:

        [alpaca]
        api_key = laskdflasdkf
        secret_key = lskdfja;lskdfjalskdj
        endpoint = https://yoyoyoyoyoy.com
        '''
        # api key attributes
        self.config = configparser.ConfigParser()
        self.config.read(keys_file)
        self.api_key = self.config['alpaca']['api_key']
        self.secret_key = self.config['alpaca']['secret_key']
        self.endpoint = self.config['alpaca']['endpoint']
        # alpaca data clients
        self.crypto_client = CryptoHistoricalDataClient()
        self.stock_client = StockHistoricalDataClient(self.api_key, self.secret_key)
        # streaming clients
        self.crypto_stream = CryptoDataStream(self.api_key, self.secret_key)
        self.stock_stream = StockDataStream(self.api_key, self.secret_key)

    def get_current_price(self, ticker):
        '''
        gets current price of specified instrument
        Parameters
        ----------
        ticker (str): ticker symbol of stock or crypto
        '''
        try:
            request_params = StockLatestQuoteRequest(symbol_or_symbols=[ticker])
            latest_quote = self.stock_client.get_stock_latest_quote(request_params)
            ask = latest_quote[ticker].ask_price
            bid = latest_quote[ticker].bid_price
            time = latest_quote[ticker].timestamp
            return time, float(bid), float(ask)
        except:
            request_params = CryptoLatestQuoteRequest(symbol_or_symbols=[ticker])
            latest_quote = self.crypto_client.get_crypto_latest_quote(request_params)
            ask = latest_quote[ticker].ask_price
            bid = latest_quote[ticker].bid_price
            time = latest_quote[ticker].timestamp
            return time, float(bid), float(ask)

    def get_historical_prices(self, ticker, start, end, freq='day'):
        '''
        gets historical prices of specified instrument
        parameters
        ----------
        ticker (str): ticker symbol of stock or crypto
        start (str): start date, e.g. "2022-06-01"
        end (str): end date, e.g. "2022-06-04"
        freq (str): frequency (e.g. "day", "hour","minute", "month", "week")
        '''
        timeframe_dict = {'day':TimeFrame.Day, 'hour':TimeFrame.Hour, 'minute':TimeFrame.Minute, 'month':TimeFrame.Month, 'week':TimeFrame.Week}
        try:
            request_params = StockBarsRequest(
                        symbol_or_symbols=[ticker],
                        timeframe=timeframe_dict[freq],
                        start=datetime.strptime(start, '%Y-%m-%d'),
                        end=datetime.strptime(end, '%Y-%m-%d')
                 )
            bars = self.stock_client.get_stock_bars(request_params)
            bars_df = bars.df
            return bars_df
        except:
            request_params = CryptoBarsRequest(
                        symbol_or_symbols=[ticker],
                        timeframe=timeframe_dict[freq],
                        start=datetime.strptime(start, '%Y-%m-%d'),
                        end=datetime.strptime(end, '%Y-%m-%d')
                 )
            bars = self.crypto_client.get_crypto_bars(request_params)
            bars_df = bars.df
            return bars_df

    async def quote_data_handler(self, data: Any):
        print(data)

    def stream_data(self, ticker):
        '''
        stream the data (NEEDS MORE WORK. WORKS FROM .py FILE.
        TRY AND EXCEPT BELOW DONT WORK AS INTENDED (OPTIONAL PARAM?))
        '''
        
        try:
            wss_client = StockDataStream(self.api_key, self.secret_key)
            wss_client.subscribe_quotes(self.quote_data_handler, ticker)
            wss_client.run()
        except:
            wss_client = CryptoDataStream(self.api_key, self.secret_key)
            wss_client.subscribe_quotes(self.quote_data_handler, ticker)
            wss_client.run()
    
    def get_account_details(self):
        '''
        returns account details like buying power,
        equity, etc.
        '''
        trading_client = TradingClient(self.api_key, self.secret_key, paper=True)
        account = trading_client.get_account()
        return account

    def get_available_assets(self, asset_class='equity', status='active', exchange='nyse'):
        '''
        get assets available for trading with Alpaca
        parameters
        ----------
        class (str): either 'equity' or 'crypto'
        status (str): avalable for trading? either 'active' or 'inactive'
        exchange (str): which exchange? 'amex', 'arca', 'bats', 'nyse', 
        'nasdaq', 'nysearca', 'ftxu', 'cbse', 'gnss', 'ersx', 'otc'
        '''
        class_dict = {'equity':AssetClass.US_EQUITY, 'crypto':AssetClass.CRYPTO}
        active_dict = {'active':AssetStatus.ACTIVE, 'inactive':AssetStatus.INACTIVE}
        exchange_dict = {'amex':AssetExchange.AMEX, 'arca':AssetExchange.ARCA, 
        'bats':AssetExchange.BATS, 'nyse':AssetExchange.NYSE, 'nasdaq':AssetExchange.NASDAQ,
        'nysearca':AssetExchange.NYSEARCA, 'ftxu':AssetExchange.FTXU, 'cbse':AssetExchange.CBSE,
        'gnss':AssetExchange.GNSS, 'ersx':AssetExchange.ERSX, 'otc':AssetExchange.OTC}

        trading_client = TradingClient(self.api_key,self.secret_key, paper=True)
        # search for crypto assets
        search_params = GetAssetsRequest(asset_class=class_dict[asset_class], 
                                        status=active_dict[status], 
                                        exchange=exchange_dict[exchange])

        assets = trading_client.get_all_assets(search_params)

        if len(assets) > 0:
            assets_df = pd.DataFrame.from_records([dict(i) for i in assets], index = [i for i in range(len(assets))])
            return assets_df
        else:
            print("no data")

    #TODO: what are the safest defaults for the below params?
    def create_order(self, ticker, units, qty_price=None, order_type='market', 
                    limit_price=None, stop_price=None, trail_price=None, trail_perc=None,
                    side='buy', time_in_force='day'):
        '''
        create either market order, limit order, stop order, stop limit order or trailing stop order
        TODO: Bracket Orders, OCO Orders, OTO Orders
        more info at: https://alpaca.markets/docs/trading/orders/
        also see: https://www.schwab.com/learn/story/3-order-types-market-limit-and-stop-orders
        Keep in mind price gaps (off hour shifts in supply and demand
        due to earnings announcements, analyst's opinion change, news release)
        parameters
        ----------
        ticker (str): ticker symbol
        units (float): number of shares to trade
        qty_price (float): dollar value of shares to trade 
        (only works with market orders. maybe limit?)
        order_type (str): type of order. e.g. 'market', 'limit', 'stop', 'stop_limit', 'trailing_stop'
        limit_price (float): limit price (hurdle for sell and the limbo for buy)
        stop_price (float): stop price (pass this point and it becomes market order 
        and executes next available price)
        trail_price (float): dollar value away from highest water mark (cum max since order submitted). 
        e.g. for sell trailing stop, stop price is hwm - trail_price. (TODO: implement)
        trail_perc (float): percent value away from hwm. 
        side (str): buy or sell? e.g. 'buy', 'sell'
        time_in_force (str): for now only 'day' (order only valid 9:30am - 4:00pm and cancelled if unfilled 
        by the end of day. IF SUBMITTED AFTER CLOSE, IT IS QUEUED AND SUBMITTED FOLLOWING DAY
        '''
        trading_client = TradingClient(self.api_key,self.secret_key, paper=True)
        buy_sell_dict = {'buy':OrderSide.BUY, 'sell':OrderSide.SELL}
        time_force_dict = {'day':TimeInForce.DAY}

        if order_type == 'market' and qty_price==None:
            # market order trade in units of number of shares
            market_order_data = MarketOrderRequest(
                                symbol=ticker,
                                qty=units,
                                side=buy_sell_dict[side],
                                time_in_force=time_force_dict[time_in_force]
                                )
            # Market Order
            market_order = trading_client.submit_order(
                            order_data=market_order_data
                        )
        elif order_type == 'market' and qty_price is not None:
            # market order trade in units of currency
            market_order_data = MarketOrderRequest(
                                symbol=ticker,
                                notional=qty_price,
                                side=buy_sell_dict[side],
                                time_in_force=time_force_dict[time_in_force]
                                )
            # Market Order
            market_order = trading_client.submit_order(
                            order_data=market_order_data
                        )
        elif order_type == 'limit':
            limit_order_data = LimitOrderRequest(
                    symbol=ticker,
                    limit_price=limit_price,
                    notional=qty_price, #how much of the share you trade (TODO: might change to qty because api says its only for market orders with stocks)
                    side=buy_sell_dict[side],
                    time_in_force=time_force_dict[time_in_force]
                   )
            # Limit Order
            limit_order = trading_client.submit_order(
                            order_data=limit_order_data
                        )
        elif order_type == 'stop':
            # the danger with these is price gaps (price can jump for a variety of reasons)
            # and you can take bigger loss than you intended (hence we have the stop limit)
            # But stop order is better in volatile market when "getting out" is the priority
            # see: https://www.schwab.com/learn/story/trading-up-close-stop-and-stop-limit-orders
            # see: https://www.investopedia.com/articles/trading/05/playinggaps.asp
            # 
            stop_order_data = StopOrderRequest(
                    symbol=ticker,
                    stop_price=stop_price,
                    qty=units,
                    side=buy_sell_dict[side],
                    time_in_force=time_force_dict[time_in_force],
                    type='stop' # look into this. you specify the order type?
                   )
            # Stop Order
            stop_order = trading_client.submit_order(
                            order_data=stop_order_data
                        )
        elif order_type == 'stop_limit':
            # can save you from the danger of price gaps but you can be screwed if price
            # falls even further. Not good for volatile market but good for end of hours
            # see the Schwab link above. 
            # also see: https://alpaca.markets/docs/trading/orders/#stop-limit-order
            stop_limit_order_data = StopLimitOrderRequest(
                    symbol=ticker,
                    stop_price=stop_price,
                    limit_price=limit_price,
                    qty=units,
                    side=buy_sell_dict[side],
                    time_in_force=time_force_dict[time_in_force],
                    type='stop_limit' # look into this. you specify the order type
                   )
            # Stop Limit Order
            stop_limit_order = trading_client.submit_order(
                            order_data=stop_limit_order_data
                        )
        elif order_type == 'trailing_stop':
            # need to make sure difference between trailing stop 
            # and price is big enough such that typical fluctuations
            # do no trigger premature execution
            trailing_stop_order_data = TrailingStopOrderRequest(
                    symbol=ticker,
                    trail_percent=trail_perc,
                    qty=units,
                    side=buy_sell_dict[side],
                    time_in_force=time_force_dict[time_in_force],
                    type='trailing_stop' # look into this. you specify the order type
                   )
            # Trailing Stop Order
            trailing_stop_order = trading_client.submit_order(
                            order_data=trailing_stop_order_data
                        )

In [60]:
trade_inst = tradeAlpaca(keys_file="../../data/alpaca_keys.cfg")

In [15]:
trade_inst.get_current_price('SPY')

(datetime.datetime(2022, 10, 28, 20, 58, 0, 695, tzinfo=datetime.timezone.utc),
 390.12,
 390.25)

In [16]:
trade_inst.get_historical_prices("BTC/USD", start="2022-01-01", end="2022-10-28").head()

open     high      low    close  \
symbol  timestamp                                                       
BTC/USD 2022-01-01 06:00:00+00:00  47218.0  47933.0  46733.0  47033.0   
        2022-01-02 06:00:00+00:00  47087.0  47985.0  46684.0  47086.0   
        2022-01-03 06:00:00+00:00  47107.0  47555.0  45709.0  46139.0   
        2022-01-04 06:00:00+00:00  46191.0  47522.0  45552.0  46310.0   
        2022-01-05 06:00:00+00:00  46310.0  47053.0  42500.0  43026.0   

                                      volume  trade_count          vwap  
symbol  timestamp                                                        
BTC/USD 2022-01-01 06:00:00+00:00   654.7013       3435.0  47287.231933  
        2022-01-02 06:00:00+00:00   782.8290       3839.0  47211.628791  
        2022-01-03 06:00:00+00:00   956.3265       3784.0  46519.493025  
        2022-01-04 06:00:00+00:00   821.2758       4670.0  46416.927260  
        2022-01-05 06:00:00+00:00  2212.1333       9279.0  44489.282863

In [17]:
# STREAMING (only uncomment when running .py file)
# ONLY WORKS FROM .py file!!!
#trade_inst.stream_data("BTC/USD")

In [18]:
trade_inst.get_account_details()

{   'account_blocked': False,
    'account_number': 'PA3RHYC1FROX',
    'accrued_fees': '0',
    'buying_power': '200000',
    'cash': '100000',
    'created_at': datetime.datetime(2022, 2, 28, 1, 19, 37, 445276, tzinfo=datetime.timezone.utc),
    'crypto_status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'currency': 'USD',
    'daytrade_count': 0,
    'daytrading_buying_power': '0',
    'equity': '100000',
    'id': UUID('833a1f64-53a1-48e6-9b45-2c687b5962bf'),
    'initial_margin': '0',
    'last_equity': '100000',
    'last_maintenance_margin': '0',
    'long_market_value': '0',
    'maintenance_margin': '0',
    'multiplier': '2',
    'non_marginable_buying_power': '100000',
    'pattern_day_trader': False,
    'pending_transfer_in': '0',
    'pending_transfer_out': None,
    'portfolio_value': '100000',
    'regt_buying_power': '200000',
    'short_market_value': '0',
    'shorting_enabled': True,
    'sma': '100000',
    'status': <AccountStatus.ACTIVE: 'ACTIVE'>,
    'trade_suspende

In [63]:
# get untradeable stocks on nasdaq
trade_inst.get_available_assets(asset_class='equity', status='inactive', exchange='nasdaq')

,id,asset_class,exchange,symbol,name,status,tradable,marginable,shortable,easy_to_borrow,fractionable,min_order_size,min_trade_increment,price_increment,maintenance_margin_requirement
0,7926ebfe-a8f7-400b-842f-b7552e3b377b,AssetClass.US_EQUITY,AssetExchange.NASDAQ,NGHC,National General Holdings Corp Common Stock,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
1,d85c6eaf-35de-4e90-b36d-5f784e6a5445,AssetClass.US_EQUITY,AssetExchange.NASDAQ,FNBG,,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
2,a84b56ae-1b6b-4b19-9de6-7e75a3db2f74,AssetClass.US_EQUITY,AssetExchange.NASDAQ,CAVM,,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
3,575e6e1b-757f-4ced-a0c2-aee48bba7e1e,AssetClass.US_EQUITY,AssetExchange.NASDAQ,NTRI,,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
4,3beec0cf-856b-4184-8b46-c112f36eb767,AssetClass.US_EQUITY,AssetExchange.NASDAQ,TCBIP,"Texas Capital Bancshares, Inc. Non Cumulative ...",AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1040,4f84e0a0-6156-43e6-850c-2e078ff4c9aa,AssetClass.US_EQUITY,AssetExchange.NASDAQ,GECCL,Great Elm Capital Corp. 6.50% Notes due 2022,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
1041,188556fd-3bd8-44a1-9977-7ae83fdddaed,AssetClass.US_EQUITY,AssetExchange.NASDAQ,HLG,Hailiang Education Group Inc. American Deposit...,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
1042,f25fb228-443d-4547-aae1-1b8e79e82d66,AssetClass.US_EQUITY,AssetExchange.NASDAQ,ITHXU,ITHAX Acquisition Corp. Unit,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0
1043,a752c8aa-e803-4658-9b62-910bff7ed08c,AssetClass.US_EQUITY,AssetExchange.NASDAQ,FSAC_DELISTED,,AssetStatus.INACTIVE,False,False,False,False,False,None,None,None,100.0


52